# Preparación de base de datos

Documentación de *pandas*:  http://pandas.pydata.org/pandas-docs/stable/

En este notebook se prepara la base de datos de indicadores con la que trabajamos en los ejemplos de análisis en *pandas*

Los indicadores con los que trabajaremos en esta sección se obtuvieron de la página del [Banco Mundial]( https://data.worldbank.org/indicator?tab=featured)

In [39]:
import pandas as pd

In [40]:
# Creamos un diccionario donde almacenaremos los metadatos de nuestra base
meta_dict = {}

Creamos un DataFrame únicamente con los nombres y códigos de los países.  A éste le iremos agregando las columnas de los indicadores

In [41]:
# Creamos una lista de las columnas que nos interesa importar
import_cols = ['Country Name', 'Country Code']

# Importamos la información del archivo csv. Indicamos que no lea los primeros tres renglones (porque están en blanco).
data = pd.read_csv('data/fertility.csv', skiprows=3, usecols=import_cols)
data.rename(columns = {'Country Name':'Country Name', 'Country Code': 'Country Code'}, inplace=True)
data.head()

,Country Name,Country Code
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO
3,Albania,ALB
4,Andorra,AND


In [42]:
# Agregamos los metadatos de los países a la base
paises = pd.read_csv('data/metadata_countries.csv', usecols=['Country Code', 'Region', 'IncomeGroup',])
paises
data = pd.merge(data, paises, on='Country Code')
data.head()

,Country Name,Country Code,Region,IncomeGroup
0,Aruba,ABW,Latin America & Caribbean,High income
1,Afghanistan,AFG,South Asia,Low income
2,Angola,AGO,Sub-Saharan Africa,Lower middle income
3,Albania,ALB,Europe & Central Asia,Upper middle income
4,Andorra,AND,Europe & Central Asia,High income


Como haremos la misma tarea varias veces, conviene crear una función que importe y agregue la columna del indicador a nuestro dataset general

In [43]:
def import_data_and_add_to_df(file_name, year, ind_name, meta_dict):
    """Dado un archivo csv y un año, esta función importa la columna correspondiente ese año,
    la agrega al dataframe indicado, y agrega los metadatos al diccionario de metadatos
    params:
        file_name: nombre del archivo (sin extensión)
        year: año del nuevo df que nos interesa agregar a data
        ind_name: nombre que tendrá el indicador en la base (asignado por nosotros)
        meta_dict: diccionario de metadatos"""
    # Usamos el df global porque nos interesa que cada vez que se llame la función se le agreguen columnas a éste
    global data
    # Asignamos las variables
    year = year
    ind_name = ind_name
    # Definimos la columnas que importaremos
    import_cols = ['Country Code', 'Indicator Name', str(year)]
    # Importamos el dataset con el indicador
    new_data = pd.read_csv('data/'+file_name+'.csv', skiprows=3, usecols=import_cols)
    # Renombramos las columnas
    new_data.rename(columns = {'Indicator Name': 'ind', str(year): ind_name}, inplace = True)
    # Agregamos los metadatos del indicador a nuestro diccionario de metadatos
    meta_dict[ind_name] = [new_data.loc[0, 'ind'],  year]
    # Eliminamos la columna innecesaria
    new_data.drop(columns = ['ind'], inplace = True)
    # Agregamos la columna del indicador-año a la nuestro dataset
    data = pd.merge(data, new_data, on='Country Code')

In [44]:
# El siguiente código se puede usar para identificar el año que nos conviene agregar a la base
# Nos interesaría el año reciente con mayor número de observaciones
file_name = 'internet_users'
pre_data = pd.read_csv(f'data/{file_name}.csv', skiprows=3,)
pre_data.isnull().sum()[-15:]

2005            20
2006            21
2007            14
2008            15
2009            15
2010            15
2011            12
2012            15
2013            16
2014            17
2015            16
2016            15
2017            14
2018           264
Unnamed: 63    264
dtype: int64

Ahora llamamos la función cada vez que le queremos agregar una columna. <br>

In [45]:
# Agregar columna de número de celulares por 100 habitantes
import_data_and_add_to_df(file_name='cellph_per100', year=2017, ind_name='cellph_per100', meta_dict = meta_dict)

# Agregar columna para emisiones de CO2 per capita
import_data_and_add_to_df(file_name = 'co2_emissions_percapita', year=2014, ind_name='co2_perc', meta_dict=meta_dict)

# Agregar columna para superficie del país
import_data_and_add_to_df(file_name='land_area', year=2017, ind_name='land_area', meta_dict=meta_dict)

# Agregar columna para acceso a electricidad
import_data_and_add_to_df(file_name='electricity_access', year=2017, ind_name='electricity' ,meta_dict=meta_dict)

# Agregar columna para fertilidad
import_data_and_add_to_df(file_name='fertility', year=2017, ind_name='fertility', meta_dict = meta_dict)

# Agregar columna para exportaciones de combustible como porcentaje de exportaciones totales
import_data_and_add_to_df(file_name='fuel_exports_percent', year=2014, ind_name='fuel_exps_pc', meta_dict = meta_dict)

# Agregar columna para días que toma para completar el proceso de empezar un negocio
import_data_and_add_to_df(file_name = 'start_business', year=2018, ind_name='start_business', meta_dict=meta_dict)

# Agregar columna para inversión extranjera directa
import_data_and_add_to_df(file_name= 'foreign_inv', year=2017, ind_name='foreign_inv', meta_dict=meta_dict)

# Agregar columna para pib
import_data_and_add_to_df(file_name = 'gdp', year=2017, ind_name='gdp', meta_dict=meta_dict)

# Agregar columna para pib per capita (ppp)
import_data_and_add_to_df(file_name='gdp_perc', year=2017, ind_name='gdp_pc', meta_dict = meta_dict)

# Agregar columna de tasa de crecimiento porcentual del pib per capita
import_data_and_add_to_df(file_name='growth_gdp_percapita', year=2017, ind_name='growth_gdp_pc', meta_dict = meta_dict)

# Agregar columna para prevalencia de VIH
import_data_and_add_to_df(file_name='hiv_prev', year=2017, ind_name='hiv_prev', meta_dict=meta_dict)

# Agregar columna para número de servidores de internet confiables por cada millón de habitantes
import_data_and_add_to_df(file_name='internet_servs_mill', year=2017, ind_name='servs_mill', meta_dict = meta_dict)

# Agregar columna para porcentaje de gasto militar 
import_data_and_add_to_df(file_name='military_expenditure', year=2018, ind_name='mil_exp', meta_dict = meta_dict)

# Agregar columna para porcentaje de la población que usa internet
import_data_and_add_to_df(file_name='internet_users', year=2017, ind_name='internet_users', meta_dict=meta_dict)

# Agregar columna para población
import_data_and_add_to_df(file_name='pop', year=2018, ind_name='population', meta_dict=meta_dict)
# Aquí había nombrado al indicador como 'pop', pero después hubo problema porque es un método para trabajar con dataframes

# Agregar columna para proporción del PIB dedicado a R&D
import_data_and_add_to_df(file_name='rd_exp_pc', year=2015, ind_name='rd_exp', meta_dict=meta_dict)

# Agregar columna para ingresos por impuestos como proporción del PIB
import_data_and_add_to_df(file_name= 'tax_revenue_pc', year=2015, ind_name='taxRev_pc', meta_dict=meta_dict)

# Agregar columna para porcentaje del PIB que aportan las actividades primarias
import_data_and_add_to_df(file_name = 'value_added_primary', year=2016, ind_name='val_primary', meta_dict=meta_dict)

# Agregar columna para porcentaje del PIB que aportan las actividades industriales (incluyendo la construcción)
import_data_and_add_to_df(file_name='val_industry', year=2016, ind_name='val_industry', meta_dict=meta_dict)

# Agregar columna para porcentaje del PIB que aportan las actividades manufactureras
import_data_and_add_to_df(file_name='val_manufacturing', year=2016, ind_name='val_manufacturing', meta_dict=meta_dict)

# Agregar columna para porcentaje del PIB que aportan los servicios
import_data_and_add_to_df(file_name='val_services', year=2016, ind_name='val_services', meta_dict=meta_dict)



In [46]:
print(data.columns)
data.head()

Index(['Country Name', 'Country Code', 'Region', 'IncomeGroup',
       'cellph_per100', 'co2_perc', 'land_area', 'electricity', 'fertility',
       'fuel_exps_pc', 'start_business', 'foreign_inv', 'gdp', 'gdp_pc',
       'growth_gdp_pc', 'hiv_prev', 'servs_mill', 'mil_exp', 'internet_users',
       'population', 'rd_exp', 'taxRev_pc', 'val_primary', 'val_industry',
       'val_manufacturing', 'val_services'],
      dtype='object')


,Country Name,Country Code,Region,IncomeGroup,cellph_per100,co2_perc,land_area,electricity,fertility,fuel_exps_pc,...,servs_mill,mil_exp,internet_users,population,rd_exp,taxRev_pc,val_primary,val_industry,val_manufacturing,val_services
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,8.410064,180.0,100.00000,1.798,0.250657,...,977.544939,NaN,97.170000,105845.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,67.350573,0.293946,652860.0,97.70000,4.477,NaN,...,43.447835,0.984561,11.447688,37172386.0,NaN,7.585382,21.081086,21.823223,11.370465,52.761719
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,44.734977,1.290307,1246700.0,41.88623,5.623,96.194403,...,10.799300,1.777138,14.339079,30809762.0,NaN,11.002019,9.831169,42.643567,6.752681,46.980994
3,Albania,ALB,Europe & Central Asia,Upper middle income,123.736096,1.978763,27400.0,100.00000,1.710,1.567523,...,443.020376,1.178901,71.847041,2866376.0,NaN,18.515790,19.849978,21.141999,5.684427,46.697169
4,Andorra,AND,Europe & Central Asia,High income,104.381212,5.832906,470.0,100.00000,NaN,0.122804,...,3415.540058,NaN,98.871436,77006.0,NaN,NaN,0.492101,9.844719,3.207886,79.204103


In [47]:
# Por comodidad, vamos a crear un código para cada región (usando map())
dict_code = {'Latin America & Caribbean':'LAm_Ca', 'South Asia':'SAs', 'Sub-Saharan Africa':'SSAf',
       'Europe & Central Asia':'Eu_CAs', 'Middle East & North Africa':'MEa_NAf',
       'East Asia & Pacific':'EAs_Pa', 'North America':'NAm'}
data['region_code'] = data.Region.map(dict_code)

# Y redefinimos los valores de 'inc_group'
dict_inc = {'High income':'high', 'Low income':'low', 'Lower middle income':'lower_mid',
       'Upper middle income':'upper_mid'}

data.IncomeGroup = data.IncomeGroup.map(dict_inc)

In [48]:
data.head()

,Country Name,Country Code,Region,IncomeGroup,cellph_per100,co2_perc,land_area,electricity,fertility,fuel_exps_pc,...,mil_exp,internet_users,population,rd_exp,taxRev_pc,val_primary,val_industry,val_manufacturing,val_services,region_code
0,Aruba,ABW,Latin America & Caribbean,high,NaN,8.410064,180.0,100.00000,1.798,0.250657,...,NaN,97.170000,105845.0,NaN,NaN,NaN,NaN,NaN,NaN,LAm_Ca
1,Afghanistan,AFG,South Asia,low,67.350573,0.293946,652860.0,97.70000,4.477,NaN,...,0.984561,11.447688,37172386.0,NaN,7.585382,21.081086,21.823223,11.370465,52.761719,SAs
2,Angola,AGO,Sub-Saharan Africa,lower_mid,44.734977,1.290307,1246700.0,41.88623,5.623,96.194403,...,1.777138,14.339079,30809762.0,NaN,11.002019,9.831169,42.643567,6.752681,46.980994,SSAf
3,Albania,ALB,Europe & Central Asia,upper_mid,123.736096,1.978763,27400.0,100.00000,1.710,1.567523,...,1.178901,71.847041,2866376.0,NaN,18.515790,19.849978,21.141999,5.684427,46.697169,Eu_CAs
4,Andorra,AND,Europe & Central Asia,high,104.381212,5.832906,470.0,100.00000,NaN,0.122804,...,NaN,98.871436,77006.0,NaN,NaN,0.492101,9.844719,3.207886,79.204103,Eu_CAs


#### Exportar datos (hay que quitar el # para hacerlo)

In [49]:
# Exportamos el dataframe a un archivo csv
# data.to_csv('data/indicadores.csv', index=False) # Ponemos index=False para que no guarde los índices como si fueran una columna

#### Exportar metadatos (hay que quitar el # para hacerlo)

In [50]:
# Creamos un dataset con los metadatos
meta_data = pd.DataFrame.from_dict(meta_dict, orient='index')
meta_data.reset_index(inplace=True)
meta_data.rename(columns={0:'descripcion', 1:'year', 'index':'indicador'}, inplace=True)
# meta_data.to_csv('data/indicadores_metadata.csv', index=False)